> 1. Rescraped all mothsofindia data with adult and early stage preclassified
> 2. Stop training when val acc not improving
> 3. Use incremental training data augmentation (random flip/jitter/crop/rotation) based on val loss/acc
> 4. Compare resnet-152 vs resnet-50

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset/moth"

In [4]:
adult_class_cnt = 0
early_class_cnt = 0
for species_dir in Path(f"{dataset_dir}/data").iterdir():
    if species_dir.is_dir() and species_dir.name.endswith('-early'):
        early_class_cnt += 1
    else:
        adult_class_cnt += 1
print(f"Adult class count: {adult_class_cnt}")
print(f"Early class count: {early_class_cnt}")

Adult class count: 3060
Early class count: 304


------------
# Older best model test result

In [5]:
model_data = torch.load(f"{dataset_dir}/checkpoint.2025.01.28.resnet152.mothsofindia.raw.25x70%+10x100%.pth", weights_only=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 3, print_preds=False, print_top1_accuracy=True)
test_top_k(model_data, f"{dataset_dir}/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 101/152 -> 66.45%, genus matched: 116/152 -> 76.32%
Top   3 accuracy: 120/152 -> 78.95%, genus matched: 134/152 -> 88.16%
Top   5 accuracy: 129/152 -> 84.87%, genus matched: 138/152 -> 90.79%
Top  10 accuracy: 133/152 -> 87.50%, genus matched: 140/152 -> 92.11%


-------------------
# A.1. Subset of data / RESNET-152

In [13]:
split_data_for_train_and_val(f"{dataset_dir}/data", 
                             f"{dataset_dir}/splits/test", f"{dataset_dir}/splits/val", f"{dataset_dir}/splits/train", 
                             test_data_weight=0.1, val_data_weight=0.2, min_file_cnt_for_val=4, 
                             class_name_filter_regex=r"^[a-b].*$")

Class count: 563
Total data count: 7462
Training data count: 5316
Validation data count: 1432
Test data count: 714


In [14]:
model_data = init_model_for_training(f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                     batch_size=32, arch="resnet152", image_size=224, robustness=0.2)

train class count: 562
val class count: 338
feature count: 2048
device: cuda:0


In [15]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet152.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 4.9768 Acc: 0.1597  | Val Loss: 4.0166 Acc: 0.2491  | Elapsed time: 0:01:29.476975
Epoch    2 /    5  | Train Loss: 2.9961 Acc: 0.3751  | Val Loss: 2.5786 Acc: 0.4367  | Elapsed time: 0:03:02.378254
Epoch    3 /    5  | Train Loss: 1.7350 Acc: 0.5730  | Val Loss: 2.2609 Acc: 0.4913  | Elapsed time: 0:04:35.874849
Epoch    4 /    5  | Train Loss: 0.9903 Acc: 0.7314  | Val Loss: 1.9076 Acc: 0.5773  | Elapsed time: 0:06:11.255499
Epoch    5 /    5  | Train Loss: 0.5720 Acc: 0.8431  | Val Loss: 2.0063 Acc: 0.6004  | Elapsed time: 0:07:46.011206


In [16]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 562
val class count: 338
0 new classes added: []
feature count: 2048
device: cuda:0


In [17]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet152.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 1.8550 Acc: 0.5476  | Val Loss: 2.0817 Acc: 0.5409  | Elapsed time: 0:01:37.310603
Epoch    2 /    5  | Train Loss: 1.3617 Acc: 0.6567  | Val Loss: 2.1331 Acc: 0.5346  | Elapsed time: 0:03:17.067274


In [18]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                    batch_size=32, image_size=224, robustness=1.0)

train class count: 562
val class count: 338
0 new classes added: []
feature count: 2048
device: cuda:0


In [19]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet152.pth", break_at_val_acc_diff=0.02)

Epoch    1 /    5  | Train Loss: 1.0073 Acc: 0.7504  | Val Loss: 1.6719 Acc: 0.6312  | Elapsed time: 0:01:34.806586
Epoch    2 /    5  | Train Loss: 0.8469 Acc: 0.7884  | Val Loss: 1.6418 Acc: 0.6445  | Elapsed time: 0:03:13.180104


In [20]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                    batch_size=32, image_size=224, robustness=2.0)

train class count: 562
val class count: 338
0 new classes added: []
feature count: 2048
device: cuda:0


In [21]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet152.pth", break_at_val_acc_diff=0.01)

Epoch    1 /    5  | Train Loss: 0.9584 Acc: 0.7547  | Val Loss: 1.5644 Acc: 0.6557  | Elapsed time: 0:01:37.917885
Epoch    2 /    5  | Train Loss: 0.8952 Acc: 0.7688  | Val Loss: 1.5491 Acc: 0.6550  | Elapsed time: 0:03:17.831415


-------------------
# A.2. Subset of data / RESNET-50

In [ ]:
split_data_for_train_and_val(f"{dataset_dir}/data", 
                             f"{dataset_dir}/splits/test", f"{dataset_dir}/splits/val", f"{dataset_dir}/splits/train", 
                             test_data_weight=0.1, val_data_weight=0.2, min_file_cnt_for_val=4, 
                             class_name_filter_regex=r"^[a-b].*$")

In [22]:
model_data = init_model_for_training(f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                     batch_size=32, arch="resnet50", image_size=224, robustness=0.2)

train class count: 562
val class count: 338
feature count: 2048
device: cuda:0


In [23]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 4.8470 Acc: 0.1810  | Val Loss: 3.7415 Acc: 0.3037  | Elapsed time: 0:01:03.664671
Epoch    2 /    5  | Train Loss: 2.5913 Acc: 0.4434  | Val Loss: 3.4489 Acc: 0.3135  | Elapsed time: 0:02:09.849688


In [24]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 562
val class count: 338
0 new classes added: []
feature count: 2048
device: cuda:0


In [25]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 2.5383 Acc: 0.4359  | Val Loss: 2.8731 Acc: 0.3961  | Elapsed time: 0:01:06.370756
Epoch    2 /    5  | Train Loss: 2.1050 Acc: 0.5211  | Val Loss: 2.3647 Acc: 0.4633  | Elapsed time: 0:02:14.084290
Epoch    3 /    5  | Train Loss: 1.8672 Acc: 0.5690  | Val Loss: 2.4005 Acc: 0.4920  | Elapsed time: 0:03:21.361272


In [26]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                    batch_size=32, image_size=224, robustness=1.0)

train class count: 562
val class count: 338
0 new classes added: []
feature count: 2048
device: cuda:0


In [27]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", break_at_val_acc_diff=0.02)

Epoch    1 /    5  | Train Loss: 1.7735 Acc: 0.5820  | Val Loss: 2.2034 Acc: 0.5115  | Elapsed time: 0:01:05.048174
Epoch    2 /    5  | Train Loss: 1.6616 Acc: 0.6072  | Val Loss: 1.9613 Acc: 0.5626  | Elapsed time: 0:02:11.802270
Epoch    3 /    5  | Train Loss: 1.2905 Acc: 0.6906  | Val Loss: 1.6487 Acc: 0.6333  | Elapsed time: 0:03:18.619823
Epoch    4 /    5  | Train Loss: 1.1620 Acc: 0.7233  | Val Loss: 1.6278 Acc: 0.6382  | Elapsed time: 0:04:25.413536


In [28]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/splits/train', f'{dataset_dir}/splits/val', 
                                    batch_size=32, image_size=224, robustness=2.0)

train class count: 562
val class count: 338
0 new classes added: []
feature count: 2048
device: cuda:0


In [29]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", break_at_val_acc_diff=0.01)

Epoch    1 /    5  | Train Loss: 1.2651 Acc: 0.6996  | Val Loss: 1.6203 Acc: 0.6508  | Elapsed time: 0:01:06.696871
Epoch    2 /    5  | Train Loss: 1.1380 Acc: 0.7289  | Val Loss: 1.6567 Acc: 0.6480  | Elapsed time: 0:02:13.532850


In [5]:
model_data = torch.load(f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", weights_only=False)

In [6]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", break_at_val_acc_diff=0.01)

Epoch    1 /    5  | Train Loss: 1.0920 Acc: 0.7389  | Val Loss: 1.5884 Acc: 0.6613  | Elapsed time: 0:01:05.241559
Epoch    2 /    5  | Train Loss: 1.0638 Acc: 0.7479  | Val Loss: 1.6001 Acc: 0.6641  | Elapsed time: 0:02:10.892381


In [7]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.subset.resnet50.pth", break_at_val_acc_diff=0.005)

Epoch    1 /    5  | Train Loss: 1.0614 Acc: 0.7472  | Val Loss: 1.5473 Acc: 0.6690  | Elapsed time: 0:01:05.373823
Epoch    2 /    5  | Train Loss: 0.9963 Acc: 0.7611  | Val Loss: 1.5577 Acc: 0.6627  | Elapsed time: 0:02:14.917107


-------------------
# B.1. All data / RESNET-152

In [5]:
model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', batch_size=32, arch="resnet152", image_size=224, robustness=0.2)

train class count: 3364
val class count: 77
feature count: 2048
device: cuda:0


In [6]:
train(model_data, 2, f"{dataset_dir}/checkpoint.moth.all.pth")

Epoch    1 /    2  | Train Loss: 7.2551 Acc: 0.0159  | Val Loss: 7.2020 Acc: 0.0000  | Elapsed time: 0:11:52.671395
Epoch    2 /    2  | Train Loss: 5.8480 Acc: 0.0958  | Val Loss: 6.0298 Acc: 0.1019  | Elapsed time: 0:23:39.239393


In [7]:
train(model_data, 8, f"{dataset_dir}/checkpoint.moth.all.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    8  | Train Loss: 3.9186 Acc: 0.2953  | Val Loss: 5.1269 Acc: 0.1529  | Elapsed time: 0:11:42.638736
Epoch    2 /    8  | Train Loss: 2.3980 Acc: 0.4946  | Val Loss: 4.3604 Acc: 0.2038  | Elapsed time: 0:23:43.985563
Epoch    3 /    8  | Train Loss: 1.5703 Acc: 0.6352  | Val Loss: 4.3406 Acc: 0.1911  | Elapsed time: 0:35:33.907545


In [8]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [9]:
train(model_data, 3, f"{dataset_dir}/checkpoint.moth.all.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    3  | Train Loss: 4.0943 Acc: 0.2233  | Val Loss: 4.5808 Acc: 0.2166  | Elapsed time: 0:12:00.497177
Epoch    2 /    3  | Train Loss: 3.6330 Acc: 0.2838  | Val Loss: 4.2664 Acc: 0.2803  | Elapsed time: 0:24:23.325251
Epoch    3 /    3  | Train Loss: 3.1738 Acc: 0.3500  | Val Loss: 4.0287 Acc: 0.2866  | Elapsed time: 0:36:43.906610


In [10]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=1.0)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [11]:
train(model_data, 3, f"{dataset_dir}/checkpoint.moth.all.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    3  | Train Loss: 3.1796 Acc: 0.3524  | Val Loss: 4.0977 Acc: 0.2611  | Elapsed time: 0:12:01.241858
Epoch    2 /    3  | Train Loss: 3.0767 Acc: 0.3718  | Val Loss: 3.9524 Acc: 0.3121  | Elapsed time: 0:24:18.471353
Epoch    3 /    3  | Train Loss: 3.0169 Acc: 0.3781  | Val Loss: 3.9728 Acc: 0.2994  | Elapsed time: 0:36:33.760276


In [ ]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=2.0)

In [ ]:
train(model_data, 3, f"{dataset_dir}/checkpoint.moth.all.pth", break_at_val_acc_diff=0.05)

-------------------
# B.2. All data / RESNET-50

In [8]:
model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', batch_size=32, arch="resnet50", image_size=224, robustness=0.2)

train class count: 3364
val class count: 77
feature count: 2048
device: cuda:0


In [9]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet50.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 6.2677 Acc: 0.0863  | Val Loss: 5.4708 Acc: 0.0955  | Elapsed time: 0:07:42.284758
Epoch    2 /    5  | Train Loss: 3.8290 Acc: 0.2928  | Val Loss: 3.7242 Acc: 0.3057  | Elapsed time: 0:15:49.543558
Epoch    3 /    5  | Train Loss: 2.1800 Acc: 0.5179  | Val Loss: 3.4832 Acc: 0.3631  | Elapsed time: 0:23:36.841112
Epoch    4 /    5  | Train Loss: 1.2167 Acc: 0.7005  | Val Loss: 3.5489 Acc: 0.3503  | Elapsed time: 0:31:20.105807


In [10]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [11]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet50.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 3.0689 Acc: 0.3726  | Val Loss: 3.6250 Acc: 0.3694  | Elapsed time: 0:08:04.857608
Epoch    2 /    5  | Train Loss: 2.5841 Acc: 0.4527  | Val Loss: 3.6225 Acc: 0.3758  | Elapsed time: 0:16:14.383217


In [12]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=1.0)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [13]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet50.pth", break_at_val_acc_diff=0.02)

Epoch    1 /    5  | Train Loss: 2.5059 Acc: 0.4675  | Val Loss: 3.7412 Acc: 0.3758  | Elapsed time: 0:07:59.326741
Epoch    2 /    5  | Train Loss: 2.0714 Acc: 0.5473  | Val Loss: 3.4152 Acc: 0.4522  | Elapsed time: 0:16:18.214413
Epoch    3 /    5  | Train Loss: 1.9222 Acc: 0.5765  | Val Loss: 3.4045 Acc: 0.4522  | Elapsed time: 0:24:26.618603


In [14]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=2.0)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [15]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet50.pth", break_at_val_acc_diff=0.01)

Epoch    1 /    5  | Train Loss: 2.1012 Acc: 0.5433  | Val Loss: 3.4946 Acc: 0.4586  | Elapsed time: 0:08:05.222918
Epoch    2 /    5  | Train Loss: 2.0074 Acc: 0.5604  | Val Loss: 3.3728 Acc: 0.4076  | Elapsed time: 0:16:12.126195


-------------------
# B.3. All data / RESNET-101

In [7]:
model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', batch_size=32, arch="resnet101", image_size=224, robustness=0.2)

train class count: 3364
val class count: 77
feature count: 2048
device: cuda:0


In [8]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.1)

Epoch    1 /    5  | Train Loss: 6.5646 Acc: 0.0581  | Val Loss: 5.8650 Acc: 0.0701  | Elapsed time: 0:10:07.799175
Epoch    2 /    5  | Train Loss: 4.3815 Acc: 0.2238  | Val Loss: 4.1190 Acc: 0.2166  | Elapsed time: 0:19:51.832664
Epoch    3 /    5  | Train Loss: 2.7922 Acc: 0.4128  | Val Loss: 3.5310 Acc: 0.2930  | Elapsed time: 0:29:24.888368


In [9]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.02)

Epoch    1 /    5  | Train Loss: 1.6961 Acc: 0.5996  | Val Loss: 3.3233 Acc: 0.3567  | Elapsed time: 0:09:37.553048
Epoch    2 /    5  | Train Loss: 0.9894 Acc: 0.7461  | Val Loss: 3.1645 Acc: 0.4204  | Elapsed time: 0:19:19.723084
Epoch    3 /    5  | Train Loss: 0.6054 Acc: 0.8380  | Val Loss: 3.6362 Acc: 0.3822  | Elapsed time: 0:29:03.860096


In [10]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [11]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.05)

Epoch    1 /    5  | Train Loss: 2.7386 Acc: 0.4242  | Val Loss: 4.0871 Acc: 0.3885  | Elapsed time: 0:10:10.156613
Epoch    2 /    5  | Train Loss: 1.9922 Acc: 0.5535  | Val Loss: 3.5272 Acc: 0.4522  | Elapsed time: 0:20:20.213853
Epoch    3 /    5  | Train Loss: 1.8260 Acc: 0.5861  | Val Loss: 3.5699 Acc: 0.4586  | Elapsed time: 0:30:32.199546


In [12]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=1)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [13]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.01)

Epoch    1 /    5  | Train Loss: 1.8969 Acc: 0.5730  | Val Loss: 3.5339 Acc: 0.4650  | Elapsed time: 0:10:13.936755
Epoch    2 /    5  | Train Loss: 1.8057 Acc: 0.5922  | Val Loss: 3.5144 Acc: 0.4650  | Elapsed time: 0:20:28.995722


In [14]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=2)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [15]:
train(model_data, 5, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.005)

Epoch    1 /    5  | Train Loss: 2.0457 Acc: 0.5463  | Val Loss: 3.6438 Acc: 0.4841  | Elapsed time: 0:09:56.973981
Epoch    2 /    5  | Train Loss: 1.9617 Acc: 0.5617  | Val Loss: 3.7655 Acc: 0.4650  | Elapsed time: 0:19:46.627363


-------------------
# B.4. All data / RESNET-101 / lr=1e-4 / weight_decay=1e-4

In [6]:
model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                     batch_size=32, arch="resnet101", image_size=224, robustness=0.2,
                                     lr=1e-4, weight_decay=1e-4)

train class count: 3364
val class count: 77
feature count: 2048
device: cuda:0


In [7]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.05)

Epoch    1 /   10  | Train Loss: 5.0013 Acc: 0.2588  | Val Loss: 2.4143 Acc: 0.5223  | Elapsed time: 0:09:46.110901
Epoch    2 /   10  | Train Loss: 1.3971 Acc: 0.7002  | Val Loss: 1.3305 Acc: 0.7006  | Elapsed time: 0:19:42.796351
Epoch    3 /   10  | Train Loss: 0.5226 Acc: 0.8714  | Val Loss: 1.3848 Acc: 0.6879  | Elapsed time: 0:29:49.464276


In [8]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [9]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.02)

Epoch    1 /   10  | Train Loss: 1.4425 Acc: 0.6791  | Val Loss: 1.5007 Acc: 0.6561  | Elapsed time: 0:10:24.610263
Epoch    2 /   10  | Train Loss: 1.1326 Acc: 0.7440  | Val Loss: 1.4989 Acc: 0.6688  | Elapsed time: 0:20:48.724198


In [10]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=1.0)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [11]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.01)

Epoch    1 /   10  | Train Loss: 1.0751 Acc: 0.7609  | Val Loss: 1.5235 Acc: 0.6688  | Elapsed time: 0:10:29.370202
Epoch    2 /   10  | Train Loss: 0.9748 Acc: 0.7783  | Val Loss: 1.4534 Acc: 0.6306  | Elapsed time: 0:21:05.792031


In [12]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=2.0)

train class count: 3364
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [13]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.all.resnet101.pth", break_at_val_acc_diff=0.005)

Epoch    1 /   10  | Train Loss: 0.8871 Acc: 0.8044  | Val Loss: 1.3888 Acc: 0.6688  | Elapsed time: 0:10:18.721319
Epoch    2 /   10  | Train Loss: 0.7811 Acc: 0.8243  | Val Loss: 1.3549 Acc: 0.6815  | Elapsed time: 0:20:34.866378
Epoch    3 /   10  | Train Loss: 0.7445 Acc: 0.8364  | Val Loss: 1.3308 Acc: 0.6879  | Elapsed time: 0:30:56.437728
Epoch    4 /   10  | Train Loss: 0.7195 Acc: 0.8429  | Val Loss: 1.3686 Acc: 0.7006  | Elapsed time: 0:41:15.316705
Epoch    5 /   10  | Train Loss: 0.6880 Acc: 0.8491  | Val Loss: 1.2834 Acc: 0.7006  | Elapsed time: 0:51:36.171163


In [12]:
model_data = torch.load(f"{dataset_dir}/checkpoint.moth.all.resnet101.tb4.ep13.pth", weights_only=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 3, print_preds=False, print_top1_accuracy=True)
test_top_k(model_data, f"{dataset_dir}/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 107/152 -> 70.39%, genus matched: 134/152 -> 88.16%
Top   3 accuracy: 136/152 -> 89.47%, genus matched: 144/152 -> 94.74%
Top   5 accuracy: 141/152 -> 92.76%, genus matched: 146/152 -> 96.05%
Top  10 accuracy: 145/152 -> 95.39%, genus matched: 146/152 -> 96.05%


-------------------
# B.5. Adult stage data / RESNET-101 / lr=1e-4 / weight_decay=1e-4

In [12]:
adult_class_cnt = 0
early_class_cnt = 0
adult_data_dir = f"{dataset_dir}/adult-data"

if os.path.exists(adult_data_dir):
        shutil.rmtree(adult_data_dir)

for species_dir in Path(f"{dataset_dir}/data").iterdir():
    if species_dir.is_dir() and species_dir.name.endswith('-early'):
        early_class_cnt += 1
    else:
        shutil.copytree(species_dir, f"{adult_data_dir}/{species_dir.name}")
        adult_class_cnt += 1

print(f"Adult class count: {adult_class_cnt}")
print(f"Early class count: {early_class_cnt}")

Adult class count: 3060
Early class count: 304


In [14]:
model_data = init_model_for_training(f'{dataset_dir}/adult-data', f'{dataset_dir}/val', 
                                     batch_size=32, arch="resnet101", image_size=224, robustness=0.2,
                                     lr=1e-4, weight_decay=1e-4)

train class count: 3060
val class count: 77
feature count: 2048
device: cuda:0


In [15]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.adult.resnet101.tb5.ep01###.pth", break_at_val_acc_diff=0.05)

Epoch    1 /   10  | Train Loss: 4.8680 Acc: 0.2715  | Val Loss: 2.5715 Acc: 0.4841  | Elapsed time: 0:08:24.062429
Epoch    2 /   10  | Train Loss: 1.4315 Acc: 0.6806  | Val Loss: 1.4375 Acc: 0.6242  | Elapsed time: 0:17:13.144536
Epoch    3 /   10  | Train Loss: 0.5212 Acc: 0.8691  | Val Loss: 1.3478 Acc: 0.7134  | Elapsed time: 0:26:05.155536
Epoch    4 /   10  | Train Loss: 0.2556 Acc: 0.9346  | Val Loss: 1.3103 Acc: 0.7197  | Elapsed time: 0:34:56.461554


In [16]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/adult-data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=0.5)

train class count: 3060
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [17]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.adult.resnet101.tb5.ep02###.pth", break_at_val_acc_diff=0.05)

Epoch    1 /   10  | Train Loss: 1.2829 Acc: 0.7143  | Val Loss: 1.6023 Acc: 0.6561  | Elapsed time: 0:09:16.569803
Epoch    2 /   10  | Train Loss: 0.9818 Acc: 0.7784  | Val Loss: 1.2800 Acc: 0.6752  | Elapsed time: 0:18:25.752396


In [18]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/adult-data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=1.0)

train class count: 3060
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [19]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.adult.resnet101.tb5.ep03###.pth", break_at_val_acc_diff=0.05)

Epoch    1 /   10  | Train Loss: 0.9466 Acc: 0.7861  | Val Loss: 1.4976 Acc: 0.6306  | Elapsed time: 0:09:13.020004
Epoch    2 /   10  | Train Loss: 0.7333 Acc: 0.8385  | Val Loss: 1.3204 Acc: 0.6943  | Elapsed time: 0:18:23.584267
Epoch    3 /   10  | Train Loss: 0.6612 Acc: 0.8548  | Val Loss: 1.2898 Acc: 0.7070  | Elapsed time: 0:28:02.805919


In [20]:
model_data = prepare_for_retraining(model_data, f'{dataset_dir}/adult-data', f'{dataset_dir}/val', 
                                    batch_size=32, image_size=224, robustness=2.0)

train class count: 3060
val class count: 77
0 new classes added: []
feature count: 2048
device: cuda:0


In [21]:
train(model_data, 10, f"{dataset_dir}/checkpoint.moth.adult.resnet101.tb5.ep04###.pth", break_at_val_acc_diff=0.05)

Epoch    1 /   10  | Train Loss: 0.7357 Acc: 0.8358  | Val Loss: 1.3163 Acc: 0.7197  | Elapsed time: 0:09:17.898283
Epoch    2 /   10  | Train Loss: 0.6888 Acc: 0.8509  | Val Loss: 1.3478 Acc: 0.7070  | Elapsed time: 0:18:28.631519


In [11]:
model_data = torch.load(f"{dataset_dir}/checkpoint.moth.adult.resnet101.tb5.ep040001.pth", weights_only=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 3, print_preds=False, print_top1_accuracy=True)
test_top_k(model_data, f"{dataset_dir}/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 10, print_preds=False, print_top1_accuracy=False)

Top   1 accuracy: 110/152 -> 72.37%, genus matched: 134/152 -> 88.16%
Top   3 accuracy: 131/152 -> 86.18%, genus matched: 141/152 -> 92.76%
Top   5 accuracy: 138/152 -> 90.79%, genus matched: 145/152 -> 95.39%
Top  10 accuracy: 144/152 -> 94.74%, genus matched: 148/152 -> 97.37%


------------------------
# Test

In [6]:
# model_data = torch.load(f"{dataset_dir}/checkpoint.moth.all.resnet101.ep13.pth", weights_only=False)
model_data = torch.load(f"{dataset_dir}/checkpoint.moth.adult.resnet101.tb5.ep040001.pth", weights_only=False)

In [7]:
test_top_k(model_data, f"{dataset_dir}/random-test", 3, print_preds=True, print_top1_accuracy=True)
test_top_k(model_data, f"{dataset_dir}/random-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/random-test", 10, print_preds=False, print_top1_accuracy=False)

acidon-nigrobasis             : miaromima-cornucopia(0.583)  westermannia-argentea(0.129)  dichromia-sagitta(0.083)  
acosmeryx-anceus              : acosmeryx-anceus(0.909)  acosmeryx-pseudonaga(0.057)  acosmeryx-sericeus(0.013)  
Adoxophyes-privatana          : adoxophyes-spp(0.940)  adoxophyes-privatana(0.025)  tortricidae-genera-spp(0.017)  
agnidra-vinacea               : agnidra-vinacea(0.937)  callidrepana-argenteola(0.032)  callidrepana-patrana(0.006)  
alcanola-speideli             : alcanola-tympanistis(0.966)  alcanola-speideli(0.018)  beana-terminigera(0.006)  
alcanola-tympanistis-2        : alcanola-tympanistis(0.469)  maliattha-spp(0.240)  nolinae-genera-spp(0.125)  
alcanola-tympanistis          : alcanola-speideli(0.888)  alcanola-tympanistis(0.075)  casminola-spp(0.017)  
angonyx-krishna-2             : daphnis-nerii(0.621)  cechenena-mirabilis(0.055)  eupanacra-busiris(0.051)  
angonyx-krishna               : angonyx-krishna(0.770)  angonyx-testacea(0.166)  callambul

In [8]:
test_top_k(model_data, f"{dataset_dir}/my-test", 3, print_preds=True, print_top1_accuracy=True)
test_top_k(model_data, f"{dataset_dir}/my-test", 5, print_preds=False, print_top1_accuracy=False)
test_top_k(model_data, f"{dataset_dir}/my-test", 10, print_preds=False, print_top1_accuracy=False)

apona-spp                     : phazaca-spp(0.159)  apona-caschmirensis(0.104)  phazaca-leucocera(0.085)  
dysphania-percota             : dysphania-percota(0.997)  nyctemera-cenis(0.001)  catocala-prolifica(0.000)  
eupterote-undata              : eupterote-undata(0.787)  eupterote-spp(0.211)  ganisa-spp(0.000)  
hippotion-rosetta-2           : hippotion-spp(0.816)  theretra-oldenlandiae(0.071)  hippotion-rosetta(0.048)  
hippotion-rosetta             : theretra-alecto(0.577)  hippotion-spp(0.267)  theretra-clotho(0.052)  
----------
Top   1 accuracy: 2/5 -> 40.00%, genus matched: 3/5 -> 60.00%
Top   3 accuracy: 3/5 -> 60.00%, genus matched: 5/5 -> 100.00%
Top   5 accuracy: 3/5 -> 60.00%, genus matched: 5/5 -> 100.00%
Top  10 accuracy: 4/5 -> 80.00%, genus matched: 5/5 -> 100.00%


In [9]:
test_top_k(model_data, f"{dataset_dir}/random-test", 10, print_preds=False, print_top1_accuracy=False, print_no_match=True)

Adoxophyes-privatana: 
	 adoxophyes-spp(0.940)  adoxophyes-privatana(0.025)  tortricidae-genera-spp(0.017)  herdonia-thaiensis(0.009)  
	 gariga-mirabilis(0.001)  choreutis-taprobanes(0.001)  galleria-mellonella(0.001)  rivula-spp(0.000)  
	 atosia-himalayana(0.000)  archips-spp(0.000)  
apona-caschmirensis: 
	 bastilla-amygdalis(0.262)  bastilla-maturescens(0.196)  pseudojana-incandescens(0.069)  poaphilini-genera-spp(0.052)  
	 ganisa-similis(0.034)  mocis-discios(0.033)  anonychia-diversilinea(0.028)  apona-shevaroyensis(0.027)  
	 hadennia-incongruens(0.023)  palirisa-cervina(0.014)  
clanis-bilineata-4: 
	 clanis-undulosa(0.202)  ampelophaga-rubiginosa(0.122)  clanis-titan(0.114)  polyptychus-trilineatus(0.113)  
	 clanis-phalaris(0.069)  clanis-hyperion(0.063)  agnosia-microta(0.044)  acosmeryx-sinjaevi(0.029)  
	 poaphilini-genera-spp(0.025)  daphnis-minima(0.024)  
clanis-bilineata: 
	 theretra-clotho(0.390)  clanis-phalaris(0.228)  nephele-hespera(0.087)  clanis-undulosa(0.083

In [10]:
test_top_k(model_data, f"{dataset_dir}/my-test", 10, print_preds=False, print_top1_accuracy=False, print_no_match=True)

apona-spp: 
	 phazaca-spp(0.159)  apona-caschmirensis(0.104)  phazaca-leucocera(0.085)  bombyx-huttoni(0.058)  
	 ambulyx-aglaia(0.049)  marumba-dyras(0.040)  epipleminae-genera-spp(0.035)  traminda-aventiaria(0.032)  
	 monobolodes-simulans(0.027)  anonychia-grisea(0.027)  
Top  10 accuracy: 4/5 -> 80.00%, genus matched: 5/5 -> 100.00%
